In [10]:
import pandas as pd
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Bishan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bishan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
df = pd.read_csv("../data/raw/reddit_emotions.csv")
df

,title,selftext,subreddit,created_utc,id,emotional_score
0,Grateful my husband doesn’t pressure me for sex,"I spent my entire life, from when I was 6 year...",gratitude,2025-06-17 16:37:52,1ldkah0,1.0
1,Grateful for getting a sleep divorce,When it comes to sleep hygiene me and my husba...,gratitude,2025-06-16 23:12:20,1lczdty,1.0
2,Grateful to get to do my shopping hauls for ot...,My partner died suddenly in 2023 and it tempor...,gratitude,2025-06-17 08:54:24,1ldcxg2,1.0
3,Grateful for my first meal after i escaped hom...,I was able to find a cheap room last weekend. ...,gratitude,2025-06-17 19:24:46,1ldnr0t,1.0
4,Gratitude didn’t “fix” my life — but it made i...,I used to roll my eyes at gratitude journals a...,gratitude,2025-06-17 14:11:07,1ldhzul,1.0
...,...,...,...,...,...,...
3616,I feel so silly,I cant understand what im feeling anymore i fe...,SuicideWatch,2025-06-15 23:10:22,1lc660y,0.0
3617,I don't want to upset my parents,"I really want everything to just end, I don't ...",SuicideWatch,2025-06-16 01:50:58,1lca0e9,0.0
3618,I want to end it all,and start over again somewhere else. Do you th...,SuicideWatch,2025-06-15 18:37:12,1lbzwd8,0.0
3619,I was supposed to be dead today,I planned four months ago to end my life today...,SuicideWatch,2025-06-16 01:39:51,1lc9qxa,0.0


In [12]:
df.isna().sum()

title              0
selftext           0
subreddit          0
created_utc        0
id                 0
emotional_score    0
dtype: int64

In [13]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if ((word not in STOPWORDS) and (word != "grateful"))]
    return " ".join(tokens)

In [14]:
df.iloc[0]['selftext']

'I spent my entire life, from when I was 6 years old, being pressured for sex. \n\nI was 6 when the first person introduced me to this pressure. That lasted until I was 12 or 13. \n\nThen I met my ex when I was 14 and spent over 20 years with him.  He wanted it daily in some way shape or form. He always complained to people that he never got it enough, even when I’d make sure that his pipes were cleaned every single day. \n\nSo ive spent most of my life with sex on my mind from the second I wake up. Not in a good way. In more of a “this will make me good enough to be loved” type of way. \n\nI am grateful that my husband has stopped watching p0rn and abusing ED medication because it’s given me such peace. I no longer have to wake up and immediately think about sex. I can just let it happen naturally, no pressure, no stress, no feeling guilty when l didn’t satisfy him that day. It has changed my view of what intimacy really is and it’s nice. \n\n'

In [15]:
df["full_text"] = df["title"].fillna("") + " " + df["selftext"].fillna("")
df["clean_text"] = df["full_text"].apply(clean_text)

In [16]:
df_clean = df[["clean_text", "subreddit", "emotional_score"]]
df_clean

,clean_text,subreddit,emotional_score
0,husband doesnt pressure sex spent entire life ...,gratitude,1.0
1,getting sleep divorce comes sleep hygiene husb...,gratitude,1.0
2,get shopping hauls others like used partner di...,gratitude,1.0
3,first meal escaped homelessness able find chea...,gratitude,1.0
4,gratitude didnt fix life made quieter inside u...,gratitude,1.0
...,...,...,...
3616,feel silly cant understand im feeling anymore ...,SuicideWatch,0.0
3617,dont want upset parents really want everything...,SuicideWatch,0.0
3618,want end start somewhere else think thats happ...,SuicideWatch,0.0
3619,supposed dead today planned four months ago en...,SuicideWatch,0.0


In [17]:
df["subreddit"].value_counts()

subreddit
TodayIAmHappy      498
BenignExistence    489
SuicideWatch       456
lonely             372
happy              363
gratitude          323
vent               308
Emotions           304
offmychest         263
Anxiety            245
Name: count, dtype: int64

In [18]:
df_clean.to_csv("../data/processed/reddit_emotions_cleaned.csv", index=False)